# 614 Project Notebook

### Members: Ayman Shahriar, Harmandeep Singh Teja, Jae Kang, Jinyu Wang, Xin Wang

### Instructor: Sarah Shah

### Link to dataset: [https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset](https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset)

# Data Collection & Wrangling

In [ ]:
# Grant colab notebook access to your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length, max, min, col, count, regexp_extract, when, lit, length, isnull, countDistinct, regexp_replace

In [ ]:
# Create SparkSession
spark = SparkSession.builder.master("local").appName("My App").getOrCreate()

In [ ]:
# Import dataset
df = spark.read.csv("/content/drive/MyDrive/100000_rows.csv", header=True, inferSchema=True, quote='"', escape='"')

In [ ]:
# Drop columns we know that will not be used
df = df.drop('vin', 'listing_id', 'main_picture_url', 'trimId', 'sp_id')

# vin, listing_id, trimId, sp_id - these columns are identification numbers. They are not meaningful data.
# main_picture_url - this is url link string. The string itself does not carry any meaning.

In [ ]:
# The dataset description mentions that this dataset includes '--' string values instead of null. Replace '--' values with null.

# print count of '--' values in columns
for col_name in df.columns:
  count_dashes = df.filter(col(col_name) == '--').count()
  if count_dashes > 0: print(f"Column '{col_name}': {count_dashes} '--' values")

# Replace "--" with null for each string column
for column_name in df.columns:
    df = df.withColumn(column_name, when(col(column_name) == "--", lit(None)).otherwise(col(column_name)))

Column 'back_legroom': 3329 '--' values
Column 'bed_height': 8123 '--' values
Column 'bed_length': 192 '--' values
Column 'front_legroom': 812 '--' values
Column 'fuel_tank_volume': 51 '--' values
Column 'height': 21 '--' values
Column 'length': 21 '--' values
Column 'maximum_seating': 22 '--' values
Column 'wheelbase': 21 '--' values
Column 'width': 22 '--' values


In [ ]:
# Review all columns

def inspect_all_columns(df):
      # Print numerical columns - show the min and max values in each column, as well as its data type
      print('Numerical Columns')
      print(f"{'Column':<25} {'Type':<20} {'Null Count':<15} {'Unique Value Count':<20} {'Min':<15} {'Max':<15}")
      print("="*100)
      numerical_cols = [col_name for col_name, dtype in df.dtypes if dtype in ('int', 'double', 'float')]

      for col_name in numerical_cols:
          col_type = df.schema[col_name].dataType
          max_value = df.agg({col_name: 'max'}).collect()[0][0]
          min_value = df.agg({col_name: 'min'}).collect()[0][0]
          null_count = df.filter(df[col_name].isNull()).count()
          unique_count = df.select(col_name).distinct().count()
          print(f"{col_name:<25} {str(col_type):<20} {null_count:<15} {unique_count:<20} {str(min_value):<15} {str(max_value):<15}")

      print("\nString Columns")
      print(f"{'Column':<25} {'Null Count':<15} {'Unique Value Count':<20} {'Min Length':<15} {'Max Length':<15}")
      print("="*80)
      string_cols = [col_name for col_name, col_type in df.dtypes if col_type == 'string']

      if string_cols:
          for col_name in string_cols:
              max_len = df.select(max(length(col(col_name)))).collect()[0][0]
              min_len = df.select(min(length(col(col_name)))).collect()[0][0]
              null_count = df.filter(df[col_name].isNull()).count()
              unique_count = df.select(col_name).distinct().count()
              print(f"{col_name:<25} {null_count:<15} {unique_count:<20} {str(min_len):<15} {str(max_len):<15}")

inspect_all_columns(df)

Numerical Columns
Column                    Type                 Null Count      Unique Value Count   Min             Max            
city_fuel_economy         DoubleType()         16016           84                   8.0             127.0          
daysonmarket              IntegerType()        0               848                  0               2150           
dealer_zip                IntegerType()        0               518                  922             98108          
engine_displacement       DoubleType()         5339            63                   700.0           8400.0         
highway_fuel_economy      DoubleType()         16016           81                   11.0            127.0          
horsepower                DoubleType()         5339            393                  70.0            808.0          
latitude                  DoubleType()         0               827                  18.3467         47.5492        
longitude                 DoubleType()         0      

In [ ]:
# Drop string columns with no value

from pyspark.sql.functions import length, col

string_cols = [col_name for col_name, col_type in df.dtypes if col_type == 'string']
for col_name in string_cols:
    max_len = df.select(max(length(col(col_name)))).collect()[0][0]
    if max_len is None:
        df = df.drop(col_name)
        print(f"Dropped column '{col_name}' due to None maximum length.")

Dropped column 'bed_height' due to None maximum length.
Dropped column 'combine_fuel_economy' due to None maximum length.
Dropped column 'is_certified' due to None maximum length.
Dropped column 'vehicle_damage_category' due to None maximum length.


In [ ]:
# Some columns should be numerical, but are string because each value includes unit string (eg. '52 in').
# We are going to convert them into numerical columns by removing units.
# Before transforming data, check if each column has a consistent unit.

columns_with_unit = ['fuel_tank_volume', 'maximum_seating','back_legroom', 'bed_length', 'front_legroom', 'height', 'length',  'wheelbase', 'width' ]

max_number_of_unit_in_each_column = 0

for column in columns_with_unit:
    # Collect unique values for the specific column
    unique_units = df.select(column).distinct().rdd.flatMap(lambda x: x).collect()

    # Extract units (assuming units are at the end of string values, separated by a space)
    units = set()
    for value in unique_units:
        if isinstance(value, str):
            parts = value.split()
            if len(parts) > 1:
                units.add(parts[-1])  # Add the last part as the potential unit
        elif value is not None:
            print(f"Non-string value found in {column}: {value}, type is {type(value)}")
    max_number_of_unit_in_each_column = max_number_of_unit_in_each_column if max_number_of_unit_in_each_column>len(units) else len(units)

    print(f"Units detected for column '{column}': {units}")

if max_number_of_unit_in_each_column<=1:
  print()
  print("Every column has consistent unit!")

Units detected for column 'fuel_tank_volume': {'gal'}
Units detected for column 'maximum_seating': {'seats'}
Units detected for column 'back_legroom': {'in'}
Units detected for column 'bed_length': {'in'}
Units detected for column 'front_legroom': {'in'}
Units detected for column 'height': {'in'}
Units detected for column 'length': {'in'}
Units detected for column 'wheelbase': {'in'}
Units detected for column 'width': {'in'}

Every column has consistent unit!


In [ ]:
# Convert every column with consistent unit into numerical column by removing unit.
columns_to_convert = ['fuel_tank_volume', 'maximum_seating', 'back_legroom', 'bed_length', 'front_legroom', 'height', 'length',  'wheelbase', 'width']

for column in columns_to_convert:
    df = df.withColumn(column, regexp_replace(column, '[a-zA-Z]+$', '')) # Remove units (assuming units are at the end, separated by a space)
    df = df.withColumn(column, col(column).cast("double")) # Convert to numerical type (double)

In [ ]:
# 'power' column consists of two values, horsepower and rpm (eg. '177 hp @ 5,750 RPM')
# Split them into two numerical columns.

df = df.withColumn('hp_at_rpm', regexp_extract(col('power'), r'(\d+)\s*hp', 1).cast('int'))
df = df.withColumn('rpm_at_hp', regexp_extract(col('power'), r'@\s*([\d,]+)\s*RPM', 1))
df = df.withColumn('rpm_at_hp', regexp_replace(col('rpm_at_hp'), ',', '').cast('int')) # Replace ',' with '' in the 'rpm' column using regexp_replace
df = df.drop('power')

In [ ]:
# 'torque' column consists of two values, torque and rpm (eg. '200 lb-ft @ 1,750 RPM')
# Split them into two numerical columns

df = df.withColumn('torque_at_rpm', regexp_extract(col('torque'), r'(\d+)\s*lb-ft', 1).cast('int'))
df = df.withColumn('rpm_at_torque', regexp_extract(col('torque'), r'@\s*([\d,]+)\s*RPM', 1))
df = df.withColumn('rpm_at_torque', regexp_replace(col('rpm_at_torque'), ',', '').cast('int')) # Replace ',' with '' in the 'torque_rpm' column using regexp_replace

df = df.drop('torque')

In [ ]:
# 'dealer_zip' column should be string, not numeric
# US zip code is 5 digits. Each digit divides a region into more granular regions.
# 5 digits divides the regions too granular. Keep first 3 digits only.

from pyspark.sql.functions import substring

df = df.withColumn("dealer_zip", col("dealer_zip").cast("string"))
df = df.withColumn("dealer_zip", substring("dealer_zip", 1, 3))

In [ ]:
inspect_all_columns(df)

Numerical Columns
Column                    Type                 Null Count      Unique Value Count   Min             Max            
back_legroom              DoubleType()         8283            188                  0.0             49.0           
bed_length                DoubleType()         92069           64                   41.5            98.6           
city_fuel_economy         DoubleType()         16016           84                   8.0             127.0          
daysonmarket              IntegerType()        0               848                  0               2150           
engine_displacement       DoubleType()         5339            63                   700.0           8400.0         
front_legroom             DoubleType()         5766            82                   35.8            67.0           
fuel_tank_volume          DoubleType()         5005            160                  1.9             63.5           
height                    DoubleType()         4975   

In [ ]:
# Minimum value of 'back_legroom' column is 0, which does not make sense. Investigate these rows.

df.filter(df.back_legroom == 0).show()

+------------+-----+----------+------------+-----------+-------------+-----------------+------------+----------+--------------------+--------------------+-------------------+--------------------+-------------------+-----+-------------+----------------+--------------+-------------+----------------+-----------------+-------------+------+--------------------+----------+----------------+-----+------+------+---------+--------+------+-----------+-------------+---------+--------------------+---------+---------------+--------+----------------+-----------+-------+-------+--------------+------------------+--------------------+-----------+------------+--------------------+--------------------+------------+--------------------+---------+-----+----+---------+---------+-------------+-------------+
|back_legroom|  bed|bed_length|   body_type|      cabin|         city|city_fuel_economy|daysonmarket|dealer_zip|         description|    engine_cylinders|engine_displacement|         engine_type|     exter

In [ ]:
# Above shows that all cars with legroom of 0 are pickup trucks (body_type column).
# Replace 0 with null.

df = df.withColumn("back_legroom", when(col("back_legroom") == 0, lit(None)).otherwise(col("back_legroom")))

In [ ]:
inspect_all_columns(df)

Numerical Columns
Column                    Type                 Null Count      Unique Value Count   Min             Max            
back_legroom              DoubleType()         8352            187                  3.5             49.0           
bed_length                DoubleType()         92069           64                   41.5            98.6           
city_fuel_economy         DoubleType()         16016           84                   8.0             127.0          
daysonmarket              IntegerType()        0               848                  0               2150           
engine_displacement       DoubleType()         5339            63                   700.0           8400.0         
front_legroom             DoubleType()         5766            82                   35.8            67.0           
fuel_tank_volume          DoubleType()         5005            160                  1.9             63.5           
height                    DoubleType()         4975   

In [ ]:
# Now let us further filter out columns that we don't need
# It seems that the values in wheel_system is just an abbreviation of the values in wheel_system_display, so we can drop wheel_system_display

df.select('wheel_system', 'wheel_system_display').distinct().show()
df = df.drop('wheel_system_display')

# The description is of type text data, not categorical data. Since we will not do any text analysis / natural language processing with this dataset, we
# can drop the description column
df = df.drop('description')

string_cols = [col_name for col_name, col_type in df.dtypes if col_type == 'string']
df[string_cols].show()

+------------+--------------------+
|wheel_system|wheel_system_display|
+------------+--------------------+
|         4WD|    Four-Wheel Drive|
|         AWD|     All-Wheel Drive|
|         4X2|                 4X2|
|         RWD|    Rear-Wheel Drive|
|         FWD|   Front-Wheel Drive|
|        NULL|                NULL|
+------------+--------------------+

+----+---------------+-----+--------+----------+----------------+-----------+--------------------+--------------+---------+--------------------+-------------+--------------------+----------+------------------+-------------------+------------+--------------------+--------------------+------------+
| bed|      body_type|cabin|    city|dealer_zip|engine_cylinders|engine_type|      exterior_color|franchise_make|fuel_type|      interior_color|listing_color|       major_options| make_name|        model_name|            sp_name|transmission|transmission_display|           trim_name|wheel_system|
+----+---------------+-----+--------+------

In [ ]:
# It seems that the engine_type and engine_cylinders hold identical values for each row, so we can get rid of one of these columns.
df.select('engine_type', 'engine_cylinders').distinct().show()
df = df.drop('engine_cylinders')

+--------------------+--------------------+
|         engine_type|    engine_cylinders|
+--------------------+--------------------+
|           I4 Diesel|           I4 Diesel|
|           I4 Hybrid|           I4 Hybrid|
|                  V8|                  V8|
|V8 Flex Fuel Vehicle|V8 Flex Fuel Vehicle|
|           I6 Diesel|           I6 Diesel|
|                  V6|                  V6|
|        V8 Biodiesel|        V8 Biodiesel|
|        V6 Biodiesel|        V6 Biodiesel|
|                  I4|                  I4|
|           V6 Diesel|           V6 Diesel|
|           V8 Diesel|           V8 Diesel|
|           V8 Hybrid|           V8 Hybrid|
|                 W12|                 W12|
|                  I2|                  I2|
|                  H6|                  H6|
|                  H4|                  H4|
|V6 Flex Fuel Vehicle|V6 Flex Fuel Vehicle|
|                  I3|                  I3|
|                  I5|                  I5|
|                 V10|          

In [ ]:
df.show()

+------------+----+----------+---------------+-----+--------+-----------------+------------+----------+-------------------+-----------+--------------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------------+-----+------+------+---------+--------+------+-----------+-------------+---------+--------------------+----------+---------------+-------+------------------+-----------+-------+-------+--------------+-------------+-------------------+-----------+------------+--------------------+--------------------+------------+---------+-----+----+---------+---------+-------------+-------------+
|back_legroom| bed|bed_length|      body_type|cabin|    city|city_fuel_economy|daysonmarket|dealer_zip|engine_displacement|engine_type|      exterior_color|fleet|frame_damaged|franchise_dealer|franchise_make|front_legroom|fuel_tank_volume|fuel_type|has_accidents|height|highway_fuel_economy|ho

In [ ]:
# We need to fill the null values
# Numerical values that are null will be set to 0
# Boolean values that are null will be set to False
# We don't need to worry about null string values because the StringIndexer in the below cell converts
# them to numerical values

df = df.fillna(0)
df = df.fillna(False)
df.show()

+------------+----+----------+---------------+-----+--------+-----------------+------------+----------+-------------------+-----------+--------------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------------+-----+------+------+---------+--------+------+-----------+-------------+---------+--------------------+----------+---------------+-------+------------------+-----------+-------+-------+--------------+-------------+-------------------+-----------+------------+--------------------+--------------------+------------+---------+-----+----+---------+---------+-------------+-------------+
|back_legroom| bed|bed_length|      body_type|cabin|    city|city_fuel_economy|daysonmarket|dealer_zip|engine_displacement|engine_type|      exterior_color|fleet|frame_damaged|franchise_dealer|franchise_make|front_legroom|fuel_tank_volume|fuel_type|has_accidents|height|highway_fuel_economy|ho

In [ ]:
# String columns cannot be used by the models of PySpark’s MLlib library, we need to encode them. In each of our string columns,
# their values have no inherent hierarchy, so they are all nominal. Nominal features can be encoded using one-hot encoding, so
# we used PySpark’s OneHotEncoder and CountVectorizer to encode them.

# Source: https://stackoverflow.com/questions/36942233/apply-stringindexer-to-several-columns-in-a-pyspark-dataframe
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

string_cols = [col_name for col_name, col_type in df.dtypes if col_type == 'string']

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid='keep').fit(df) for column in string_cols]
pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df)
df_indexed = df_indexed.drop(*string_cols)
df_indexed.show()

+------------+----------+-----------------+------------+-------------------+-----+-------------+----------------+-------------+----------------+-------------+------+--------------------+----------+-----+------+------+---------+--------+------+-----------+---------+---------------+-------+-----------+-------+-------+--------------+-------------+-----------+---------+-----+----+---------+---------+-------------+-------------+---------+---------------+-----------+----------+----------------+-----------------+--------------------+--------------------+---------------+--------------------+-------------------+-------------------+---------------+----------------+-------------+------------------+--------------------------+---------------+------------------+
|back_legroom|bed_length|city_fuel_economy|daysonmarket|engine_displacement|fleet|frame_damaged|franchise_dealer|front_legroom|fuel_tank_volume|has_accidents|height|highway_fuel_economy|horsepower|isCab|is_cpo|is_new|is_oemcpo|latitude|lengt

In [ ]:
from pyspark.ml.feature import OneHotEncoder

indexed_cols = list(map(lambda x : x + '_index', string_cols))

encoders = [OneHotEncoder(inputCol=column, outputCol=column+'_onehot', handleInvalid='keep').fit(df_indexed) for column in indexed_cols]
pipeline = Pipeline(stages=encoders)
df_encoded = pipeline.fit(df_indexed).transform(df_indexed)
df_encoded = df_encoded.drop(*indexed_cols)
df_encoded.show()

+------------+----------+-----------------+------------+-------------------+-----+-------------+----------------+-------------+----------------+-------------+------+--------------------+----------+-----+------+------+---------+--------+------+-----------+---------+---------------+-------+-----------+-------+-------+--------------+-------------+-----------+---------+-----+----+---------+---------+-------------+-------------+----------------+----------------------+------------------+-----------------+-----------------------+------------------------+---------------------------+---------------------------+----------------------+---------------------------+--------------------------+--------------------------+----------------------+-----------------------+--------------------+-------------------------+---------------------------------+----------------------+-------------------------+
|back_legroom|bed_length|city_fuel_economy|daysonmarket|engine_displacement|fleet|frame_damaged|franchise_de

In [ ]:
# Convert date to unix timestamp
from pyspark.sql.functions import unix_timestamp
df_encoded = df_encoded.withColumn("listed_date", unix_timestamp("listed_date", "yyyy-MM-dd"))

In [ ]:
# pyspark.ml models require the data to be transformed using a VectorAssembler
from pyspark.ml.feature import VectorAssembler

feature_names = df_encoded.drop(*(string_cols + indexed_cols + ['price'])).columns
assembler = VectorAssembler(inputCols=feature_names, outputCol='features', handleInvalid='keep')

df_assembler = assembler.transform(df_encoded)
df_assembler.show()


+------------+----------+-----------------+------------+-------------------+-----+-------------+----------------+-------------+----------------+-------------+------+--------------------+----------+-----+------+------+---------+--------+------+-----------+---------+---------------+-------+-----------+-------+-------+--------------+-------------+-----------+---------+-----+----+---------+---------+-------------+-------------+----------------+----------------------+------------------+-----------------+-----------------------+------------------------+---------------------------+---------------------------+----------------------+---------------------------+--------------------------+--------------------------+----------------------+-----------------------+--------------------+-------------------------+---------------------------------+----------------------+-------------------------+--------------------+
|back_legroom|bed_length|city_fuel_economy|daysonmarket|engine_displacement|fleet|frame

# Model Building and Hyperparameter Tuning

In [ ]:
# Let us build a linear regression model for this data
from pyspark.ml.regression import LinearRegression

data = df_assembler.select('features', 'price')
train_data, test_data = data.randomSplit([0.75, 0.25], seed=42)
lr_model = LinearRegression(featuresCol='features', labelCol='price', predictionCol='prediction')

# fit linear regression model to data
lr_model = lr_model.fit(train_data)

In [ ]:
# Print out the train and test results of our linear regression model

lr_train_rmse = lr_model.summary.rootMeanSquaredError
lr_train_r2 = lr_model.summary.r2
print("Root Mean Squared Error (RMSE) on training data: {:.3f}".format(lr_train_rmse))
print("R-squared (R2) on training data: {:.3f}".format(lr_train_r2))
print('\n')

# Source: https://www.machinelearningplus.com/pyspark/pyspark-linear-regression/
from pyspark.ml.evaluation import RegressionEvaluator

predictions = lr_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
lr_test_rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(lr_test_rmse))

evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
lr_test_r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2) on test data: {:.3f}".format(lr_test_r2))

Root Mean Squared Error (RMSE) on training data: 2512.376
R-squared (R2) on training data: 0.984


Root Mean Squared Error (RMSE) on test data: 12790.756
R-squared (R2) on test data: 0.691


In [ ]:
# It seems that our linear regression model is overfitting on the training data, since there is a large difference between the train
# and test r2 scores.
# Let's try out a linear model that does regularization, such as lasso regression.
# We will carry out hyperparameter tuning (using grid search and cross validation), to find the most optimal value for the L1 regularization parameter.
# Source: https://www.machinelearningplus.com/pyspark/pyspark-lasso-regression/

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

lasso_regression = LinearRegression(featuresCol='features', labelCol='price', predictionCol='prediction', elasticNetParam=1)

# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(lasso_regression.regParam, [1.0, 10, 100, 1000]) \
    .build()

# Create the cross-validator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol= 'price', metricName="r2")
cross_validator = CrossValidator(estimator=lasso_regression,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=5)

# Train the model with the best hyperparameters
cv_model = cross_validator.fit(train_data)
lasso_model = cv_model.bestModel

print("Optimal regularization parameter: ", lasso_model.getRegParam())

Optimal regularization parameter:  10.0


In [ ]:
# Print out the train and test results of our ridge regression model

lasso_train_rmse = lasso_model.summary.rootMeanSquaredError
lasso_train_r2 = lasso_model.summary.r2
print("Root Mean Squared Error (RMSE) on training data: {:.3f}".format(lasso_train_rmse))
print("R-squared (R2) on training data: {:.3f}".format(lasso_train_r2))
print('\n')

predictions = lasso_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
lasso_test_rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(lasso_test_rmse))

evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
lasso_test_r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2) on test data: {:.3f}".format(lasso_test_r2))

Root Mean Squared Error (RMSE) on training data: 2988.879
R-squared (R2) on training data: 0.978


Root Mean Squared Error (RMSE) on test data: 12361.408
R-squared (R2) on test data: 0.711


In [ ]:
# Now let's build a decision tree model
# We will carry out hyperparameter tuning (using grid search and cross validation), to find the optimal maximum depth of the decision tree.

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

dt_regression = DecisionTreeRegressor(featuresCol='features', labelCol='price', predictionCol='prediction')

# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(dt_regression.maxDepth, [7, 8, 9, 10, 11]) \
    .build()

# Create the cross-validator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol= 'price', metricName="r2")
cross_validator = CrossValidator(estimator=dt_regression,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=5)

# Train the model with the best hyperparameters
cv_model = cross_validator.fit(train_data)
dt_model = cv_model.bestModel

print("Optimal maximum depth of decision tree: ", dt_model.getMaxDepth())

Optimal maximum depth of decision tree:  11


In [ ]:
# Print out the train and test results of the decision tree

train_predictions = dt_model.transform(train_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
dt_train_rmse = evaluator.evaluate(train_predictions)
print("Root Mean Squared Error (RMSE) on training data: {:.3f}".format(dt_train_rmse))

evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
dt_train_r2 = evaluator_r2.evaluate(train_predictions)
print("R-squared (R2) on training data: {:.3f}".format(dt_train_r2))

print()

predictions = dt_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
dt_test_rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(dt_test_rmse))

evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
dt_test_r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2) on test data: {:.3f}".format(dt_test_r2))


Root Mean Squared Error (RMSE) on training data: 4944.755
R-squared (R2) on training data: 0.940

Root Mean Squared Error (RMSE) on test data: 13050.859
R-squared (R2) on test data: 0.678


# Summary of Model Results

In [ ]:
# Summarize the validation metrics of each model
import pandas as pd
pd.options.display.float_format = "{:,.3f}".format

r2_scores_as_dict = {
    'Linear Regression': [lr_train_r2, lr_test_r2],
    'Lasso Regression': [lasso_train_r2, lasso_test_r2],
    'Decision Tree': [dt_train_r2, dt_test_r2],
}

r2_scores = pd.DataFrame(r2_scores_as_dict, index=['Train Score', 'Test Score'])
print('R2 Scores:')
display(r2_scores)

print('\n\n')

rmse_scores_as_dict = {
    'Linear Regression': [lr_train_rmse, lr_test_rmse],
    'Lasso Regression': [lasso_train_rmse, lasso_test_rmse],
    'Decision Tree': [dt_train_rmse, dt_test_rmse],
}

rmse_scores = pd.DataFrame(rmse_scores_as_dict, index=['Train Score', 'Test Score'])
print('RMSE Scores:')
display(rmse_scores)


R2 Scores:


,Linear Regression,Lasso Regression,Decision Tree
Train Score,0.984,0.978,0.940
Test Score,0.691,0.711,0.678





RMSE Scores:


,Linear Regression,Lasso Regression,Decision Tree
Train Score,"2,512.376","2,988.879","4,944.755"
Test Score,"12,790.756","12,361.408","13,050.859"
